In [1]:
import os, sys
sys.path.insert(1, "/".join(os.path.abspath('').split("/")[:-1]))

import torch
from transformer.multi_head_attention.attention_mechanism import AttentionMechanism
from transformer.multi_head_attention.attention_mechanism.attn_params import CosformerParams, PerformerParams, VanillaParams


/Users/cezarygolecki/projects/magisterka/.venv/lib/python3.11/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:35: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/Users/cezarygolecki/projects/magisterka/.venv/lib/python3.11/site-packages/rotary_embedding_torch/rotary_embedding_torch.py:268: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)


In [2]:
sum([n**2 for n in range(5, 10)])

255

In [9]:
# DOWNSAMPLING
from torch import nn

batch_size = 2
seq_len = 6
factor = 2
d_model = 5

q = torch.arange(0, batch_size * seq_len//factor * d_model).reshape(batch_size, seq_len//factor, d_model)
k = torch.arange(0, batch_size * seq_len * d_model).reshape(batch_size, seq_len, d_model)
v = torch.arange(0, batch_size * seq_len * d_model).reshape(batch_size, seq_len, d_model)
print(q.shape, k.shape, v.shape)
k = k.view(
            batch_size,
            seq_len // factor,
            factor,
            d_model,
            )
weights = torch.einsum("bsd,bsfd->bsf", q, k)
print(weights.shape)
weights = weights.flatten(1)
print(weights.shape)
out = torch.einsum("bs,bsd->bsd", weights, v)
print(out)
avg_pool = nn.AvgPool1d(kernel_size=factor, stride=factor)

torch.Size([2, 3, 5]) torch.Size([2, 6, 5]) torch.Size([2, 6, 5])
torch.Size([2, 3, 2])
torch.Size([2, 6])
tensor([[[     0,     30,     60,     90,    120],
         [   400,    480,    560,    640,    720],
         [  4300,   4730,   5160,   5590,   6020],
         [  9075,   9680,  10285,  10890,  11495],
         [ 26600,  27930,  29260,  30590,  31920],
         [ 40750,  42380,  44010,  45640,  47270]],

        [[ 81900,  84630,  87360,  90090,  92820],
         [110425, 113580, 116735, 119890, 123045],
         [185200, 189830, 194460, 199090, 203720],
         [233100, 238280, 243460, 248640, 253820],
         [351500, 358530, 365560, 372590, 379620],
         [423775, 431480, 439185, 446890, 454595]]])


In [5]:
# UPSAMPLING
batch_size = 2
seq_len = 6
factor = 2
d_model = 5

q = torch.arange(0, batch_size * seq_len * d_model).reshape(batch_size, seq_len, d_model)
k = torch.arange(0, batch_size * seq_len//factor * d_model).reshape(batch_size, seq_len//factor, d_model)
v = torch.arange(0, batch_size * seq_len//factor * d_model).reshape(batch_size, seq_len//factor, d_model)
print(q.shape, k.shape, v.shape)
q = q.view(
            batch_size,
            seq_len // factor,
            factor,
            d_model,
            )
weights = torch.einsum("bsfd,bsd->bsf", q, k)
print(weights)
print(v)
a = torch.einsum("bsf,bsd->bsfd", weights, v)
print(a.view(batch_size, seq_len, d_model).shape)

torch.Size([2, 6, 5]) torch.Size([2, 3, 5]) torch.Size([2, 3, 5])
tensor([[[  30,   80],
         [ 430,  605],
         [1330, 1630]],

        [[2730, 3155],
         [4630, 5180],
         [7030, 7705]]])
tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14]],

        [[15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]]])
torch.Size([2, 6, 5])


In [20]:
# UPSAMPLING
batch_size = 2
seq_len = 6
factor = 2
d_model = 5

q = torch.arange(0, batch_size * seq_len * d_model).reshape(batch_size, seq_len//factor, factor, d_model)
k = torch.arange(0, batch_size * seq_len//factor * d_model).reshape(batch_size, seq_len//factor, d_model)

print(q)
print(k)

weights = torch.einsum("bsfd,bsd->bsf", q, k)

print(weights)

tensor([[[[ 0,  1,  2,  3,  4],
          [ 5,  6,  7,  8,  9]],

         [[10, 11, 12, 13, 14],
          [15, 16, 17, 18, 19]],

         [[20, 21, 22, 23, 24],
          [25, 26, 27, 28, 29]]],


        [[[30, 31, 32, 33, 34],
          [35, 36, 37, 38, 39]],

         [[40, 41, 42, 43, 44],
          [45, 46, 47, 48, 49]],

         [[50, 51, 52, 53, 54],
          [55, 56, 57, 58, 59]]]])
tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14]],

        [[15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29]]])
tensor([[[  30,   80],
         [ 430,  605],
         [1330, 1630]],

        [[2730, 3155],
         [4630, 5180],
         [7030, 7705]]])


In [2]:
batch_size = 2
seq_len = 5
d_model = 6
n_heads = 2
eps = 0.00000001
query = key = value = torch.rand((batch_size, n_heads, seq_len, d_model//n_heads))

# Vanilla

In [3]:
batch_size = 2
seq_len = 6
embed_dim = 8
device = "cpu"
x = torch.rand((batch_size, seq_len, embed_dim)).to(device)
v = torch.rand((batch_size, 200, embed_dim)).to(device)

attn = AttentionMechanism(embed_dim, 4, VanillaParams(), False, device)
print(attn(x,x,x, causal=True))

tensor([[[0.0967, 0.4586, 0.5522, 0.7956, 0.8590, 0.3303, 0.9981, 0.4314],
         [0.1161, 0.5292, 0.7440, 0.5752, 0.9253, 0.2856, 0.7630, 0.3724],
         [0.2382, 0.3825, 0.8475, 0.7594, 0.7898, 0.2678, 0.6966, 0.5597],
         [0.3342, 0.3822, 0.7424, 0.6594, 0.7903, 0.2991, 0.5103, 0.5450],
         [0.4650, 0.4563, 0.7720, 0.6581, 0.7684, 0.3595, 0.5652, 0.4799],
         [0.5582, 0.3749, 0.7021, 0.6443, 0.6653, 0.3863, 0.5724, 0.4100]],

        [[0.5541, 0.1670, 0.8828, 0.0428, 0.3483, 0.3645, 0.4024, 0.8520],
         [0.7369, 0.3503, 0.5351, 0.6196, 0.4813, 0.7096, 0.7302, 0.3852],
         [0.7193, 0.3262, 0.6049, 0.4748, 0.5696, 0.6768, 0.6764, 0.4246],
         [0.5083, 0.4979, 0.5489, 0.5324, 0.6331, 0.5941, 0.6304, 0.3710],
         [0.6635, 0.4389, 0.5138, 0.5449, 0.5514, 0.6653, 0.6732, 0.3411],
         [0.7344, 0.4274, 0.5199, 0.4742, 0.5061, 0.5453, 0.6727, 0.4627]]])


# Cosformer

In [30]:
batch_size = 1
seq_len = 5
embed_dim = 6
num_heads = 3
device = "cpu"

from transformer.layers.multi_head_attention.attention_mechanism.cosformer import Cosformer

x = torch.rand((batch_size, seq_len, embed_dim)).to(device)
v = torch.rand((batch_size, seq_len, embed_dim)).to(device)

attn = AttentionMechanism(embed_dim, num_heads, CosformerParams(), False, device)
print(attn(x,v,v, causal=True))


x = torch.rand((batch_size, num_heads, seq_len, embed_dim//num_heads)).to(device)
v = torch.rand((batch_size, num_heads, seq_len, embed_dim//num_heads)).to(device)

a = Cosformer(embed_dim, num_heads, 1e-8, device)

tensor([[[0.0389, 0.7063, 0.2557, 0.5169, 0.8304, 0.0929],
         [0.2956, 0.5350, 0.5315, 0.4639, 0.7503, 0.1433],
         [0.2882, 0.6009, 0.5320, 0.4246, 0.3359, 0.3572],
         [0.4696, 0.4673, 0.5108, 0.6043, 0.5944, 0.4020],
         [0.4350, 0.6297, 0.3256, 0.7654, 0.6061, 0.5014]]])
torch.Size([1, 3, 5, 2])
tensor([[[True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True],
         [True, True, True, True, True, True]]])


# Performer

In [5]:
batch_size = 2
seq_len = 5
embed_dim = 4
device = "cpu"

x = torch.rand((batch_size, seq_len, embed_dim)).to(device)
v = torch.rand((batch_size,  seq_len, embed_dim)).to(device)

attn = AttentionMechanism(embed_dim, 2, PerformerParams(),False, device)
attn(x,x,v, causal=True)

tensor([[[0.5286, 0.1363, 0.4103, 0.4354],
         [0.7134, 0.5988, 0.6635, 1.0399],
         [0.8281, 1.2339, 0.8651, 1.1266],
         [1.4964, 2.3034, 1.3261, 1.5997],
         [1.5555, 2.9737, 1.8209, 1.8884]],

        [[0.1080, 0.3290, 0.3216, 0.0357],
         [0.5594, 1.1571, 0.7290, 0.4924],
         [1.0235, 1.5367, 0.7983, 0.5686],
         [1.1612, 1.8700, 0.7925, 0.5744],
         [2.1675, 2.4708, 1.0163, 1.0277]]])